# Premissas do Algoritmo KNN (K-Nearest Neighbors)

O KNN é um algoritmo de aprendizado supervisionado baseado em instâncias. Ele classifica ou faz previsões com base nos K vizinhos mais próximos.

## Premissa 1: Similaridade entre Observações

**Princípio:** Observações similares tendem a ter comportamentos similares.

O KNN assume que pontos próximos no espaço de características compartilham a mesma classe ou valor.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Gerar dados sintéticos com 3 clusters
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=42)

# Visualizar
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50, edgecolor='k')
plt.title('Pontos próximos tendem a pertencer à mesma classe')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.show()

## Premissa 2: Métrica de Distância Adequada

**Princípio:** A escolha da métrica de distância afeta diretamente a definição de "proximidade".

Principais métricas:
- **Euclidiana:** $d = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$ - Distância em linha reta
- **Manhattan:** $d = \sum_{i=1}^{n}|x_i - y_i|$ - Distância por caminhos ortogonais
- **Minkowski:** Generalização das anteriores

In [ ]:
# Comparação de métricas de distância
ponto_A = np.array([1, 1])
ponto_B = np.array([4, 5])

# Distância Euclidiana
dist_euclidiana = np.sqrt(np.sum((ponto_A - ponto_B)**2))

# Distância Manhattan
dist_manhattan = np.sum(np.abs(ponto_A - ponto_B))

print(f"Ponto A: {ponto_A}")
print(f"Ponto B: {ponto_B}")
print(f"\nDistância Euclidiana: {dist_euclidiana:.2f}")
print(f"Distância Manhattan: {dist_manhattan:.2f}")

# Visualizar as diferenças
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Euclidiana
ax[0].plot([ponto_A[0], ponto_B[0]], [ponto_A[1], ponto_B[1]], 'r-', linewidth=2, label='Euclidiana')
ax[0].scatter(*ponto_A, s=200, c='blue', edgecolor='k', zorder=3)
ax[0].scatter(*ponto_B, s=200, c='green', edgecolor='k', zorder=3)
ax[0].set_title('Distância Euclidiana')
ax[0].grid(True, alpha=0.3)
ax[0].legend()

# Manhattan
ax[1].plot([ponto_A[0], ponto_B[0]], [ponto_A[1], ponto_A[1]], 'b-', linewidth=2)
ax[1].plot([ponto_B[0], ponto_B[0]], [ponto_A[1], ponto_B[1]], 'b-', linewidth=2, label='Manhattan')
ax[1].scatter(*ponto_A, s=200, c='blue', edgecolor='k', zorder=3)
ax[1].scatter(*ponto_B, s=200, c='green', edgecolor='k', zorder=3)
ax[1].set_title('Distância Manhattan')
ax[1].grid(True, alpha=0.3)
ax[1].legend()

plt.tight_layout()
plt.show()

## Premissa 3: Normalização das Features

**Princípio:** Features em escalas diferentes podem dominar o cálculo de distância.

**Problema:** Uma feature com valores entre 0-1000 terá peso muito maior que uma feature entre 0-1.

**Solução:** Normalizar ou padronizar os dados antes de aplicar o KNN.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Dados com escalas diferentes
idade = np.array([25, 30, 35]).reshape(-1, 1)
salario = np.array([30000, 45000, 60000]).reshape(-1, 1)

# Combinar features
dados = np.hstack([idade, salario])

# Novo ponto para avaliar
ponto_novo = np.array([[28, 35000]])

# SEM normalização
dist_sem_norm = np.sqrt(np.sum((dados - ponto_novo)**2, axis=1))

# COM normalização
scaler = StandardScaler()
dados_norm = scaler.fit_transform(dados)
ponto_novo_norm = scaler.transform(ponto_novo)
dist_com_norm = np.sqrt(np.sum((dados_norm - ponto_novo_norm)**2, axis=1))

print("Dados originais:")
print("Idade | Salário")
for i in range(len(dados)):
    print(f"{dados[i, 0]:5.0f} | {dados[i, 1]:7.0f}")
print(f"\nPonto novo: [{ponto_novo[0, 0]:.0f}, {ponto_novo[0, 1]:.0f}]")

print("\n--- SEM Normalização ---")
print(f"Distâncias: {dist_sem_norm}")
print(f"Vizinho mais próximo: índice {np.argmin(dist_sem_norm)}")

print("\n--- COM Normalização ---")
print(f"Distâncias: {dist_com_norm}")
print(f"Vizinho mais próximo: índice {np.argmin(dist_com_norm)}")

print("\nNota: A diferença de escala faz o salário dominar o cálculo sem normalização.")

In [ ]:
# Visualizar o impacto da normalização
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Dados SEM normalização
ax1 = axes[0]
ax1.scatter(dados[:, 0], dados[:, 1], s=150, c='blue', edgecolor='k', label='Dados treino', alpha=0.7)
ax1.scatter(ponto_novo[0, 0], ponto_novo[0, 1], s=200, c='red', marker='*', edgecolor='k', label='Ponto novo', zorder=5)

# Desenhar círculos de distância sem normalização
for i, dist in enumerate(dist_sem_norm):
    circle = plt.Circle((ponto_novo[0, 0], ponto_novo[0, 1]), dist, fill=False, linestyle='--', alpha=0.5)
    ax1.add_patch(circle)
    ax1.plot([ponto_novo[0, 0], dados[i, 0]], [ponto_novo[0, 1], dados[i, 1]], 'k--', alpha=0.3)
    ax1.text(dados[i, 0], dados[i, 1] + 2000, f'd={dist:.0f}', fontsize=9, ha='center')

ax1.set_xlabel('Idade', fontsize=11)
ax1.set_ylabel('Salário', fontsize=11)
ax1.set_title('SEM Normalização\n(Salário domina o cálculo)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Dados COM normalização
ax2 = axes[1]
ax2.scatter(dados_norm[:, 0], dados_norm[:, 1], s=150, c='blue', edgecolor='k', label='Dados treino', alpha=0.7)
ax2.scatter(ponto_novo_norm[0, 0], ponto_novo_norm[0, 1], s=200, c='red', marker='*', edgecolor='k', label='Ponto novo', zorder=5)

# Desenhar círculos de distância com normalização
for i, dist in enumerate(dist_com_norm):
    circle = plt.Circle((ponto_novo_norm[0, 0], ponto_novo_norm[0, 1]), dist, fill=False, linestyle='--', alpha=0.5)
    ax2.add_patch(circle)
    ax2.plot([ponto_novo_norm[0, 0], dados_norm[i, 0]], [ponto_novo_norm[0, 1], dados_norm[i, 1]], 'k--', alpha=0.3)
    ax2.text(dados_norm[i, 0], dados_norm[i, 1] + 0.3, f'd={dist:.2f}', fontsize=9, ha='center')

ax2.set_xlabel('Idade (normalizada)', fontsize=11)
ax2.set_ylabel('Salário (normalizado)', fontsize=11)
ax2.set_title('COM Normalização\n(Features equilibradas)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservação:")
print("- À esquerda: diferença enorme de escala faz o salário dominar completamente")
print("- À direita: após normalização, ambas as features contribuem igualmente para a distância")

## Premissa 4: Escolha Adequada do K

**Princípio:** O valor de K afeta diretamente o viés e a variância do modelo.

- **K pequeno (ex: K=1):** Modelo mais complexo, sensível a ruídos (alta variância)
- **K grande:** Modelo mais simples, pode perder padrões locais (alto viés)
- **Regra prática:** K costuma ser ímpar para evitar empates em classificação binária

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Gerar dados com sobreposição para que diferenças em K sejam mais visíveis
np.random.seed(42)
X1 = np.random.randn(60, 2) + np.array([2, 2])
X2 = np.random.randn(60, 2) + np.array([-2, -2])
X3 = np.random.randn(60, 2) + np.array([2, -2])
X = np.vstack([X1, X2, X3])
y = np.hstack([np.zeros(60), np.ones(60), np.full(60, 2)])

# Visualizar fronteiras de decisão para diferentes valores de K
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
valores_k = [1, 3, 7, 15, 30, 60]

for idx, k in enumerate(valores_k):
    ax = axes[idx // 3, idx % 3]
    
    # Treinar modelo
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y)
    
    # Criar mesh para visualizar fronteira de decisão
    h = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plotar fronteira com cores mais definidas
    ax.contourf(xx, yy, Z, alpha=0.4, cmap='RdYlBu', levels=2)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', s=50, edgecolor='black', linewidth=0.5, alpha=0.9)
    ax.set_title(f'K = {k}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Feature 1', fontsize=11)
    ax.set_ylabel('Feature 2', fontsize=11)
    ax.grid(True, alpha=0.2)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

print("Análise das fronteiras:")
print("- K=1: Fronteira extremamente irregular com 'ilhas' (sobreajuste)")
print("- K=3 a K=7: Fronteira razoavelmente suave, captura padrões reais")
print("- K=15: Fronteira mais suave, começa a perder detalhes")
print("- K=30 e K=60: Fronteira muito simplificada (subajuste)")

### Como identificar o melhor K?

Use **validação cruzada** para testar diferentes valores de K e escolher aquele com melhor desempenho.

In [ ]:
from sklearn.model_selection import cross_val_score

# Separar dados para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Testar valores de K de 1 a 50
k_range = range(1, 51)
scores_mean = []
scores_std = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    # Validação cruzada com 5 folds
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    scores_mean.append(scores.mean())
    scores_std.append(scores.std())

# Encontrar o melhor K
melhor_k = k_range[np.argmax(scores_mean)]
melhor_score = max(scores_mean)

# Visualizar
plt.figure(figsize=(12, 6))
plt.plot(k_range, scores_mean, 'b-', linewidth=2, label='Acurácia média')
plt.fill_between(k_range, 
                 np.array(scores_mean) - np.array(scores_std),
                 np.array(scores_mean) + np.array(scores_std),
                 alpha=0.2, color='blue', label='± 1 desvio padrão')
plt.axvline(x=melhor_k, color='red', linestyle='--', linewidth=2, label=f'Melhor K = {melhor_k}')
plt.xlabel('Valor de K', fontsize=12)
plt.ylabel('Acurácia (validação cruzada)', fontsize=12)
plt.title('Identificando o Melhor Valor de K', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 51, 5))
plt.show()

print(f"Melhor K: {melhor_k}")
print(f"Acurácia média (validação cruzada): {melhor_score:.3f}")
print(f"\nDica: Prefira valores ímpares para evitar empates em classificação binária")

## Premissa 5: Dados Balanceados

**Princípio:** Classes desbalanceadas podem enviesar as previsões.

Se uma classe tem muito mais exemplos que outra, o KNN tende a favorecer a classe majoritária. Em casos de desbalanceamento severo, considere:
- Reamostragem (oversampling/undersampling)
- Pesos diferentes para as classes
- Usar versões ponderadas do KNN

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter

# Criar dataset desbalanceado (90% classe 0, 10% classe 1)
X_desb, y_desb = make_classification(n_samples=200, n_features=2, n_informative=2, 
                                      n_redundant=0, n_clusters_per_class=1,
                                      weights=[0.9, 0.1], random_state=42)

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_desb, y_desb, test_size=0.3, random_state=42)

print("Distribuição das classes no treino:")
print(Counter(y_train_d))

# KNN sem ponderação
knn_normal = KNeighborsClassifier(n_neighbors=5)
knn_normal.fit(X_train_d, y_train_d)
y_pred_normal = knn_normal.predict(X_test_d)

# KNN com ponderação por distância
knn_ponderado = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_ponderado.fit(X_train_d, y_train_d)
y_pred_ponderado = knn_ponderado.predict(X_test_d)

print("\n--- Resultados ---")
print(f"Acurácia KNN normal: {accuracy_score(y_test_d, y_pred_normal):.3f}")
print(f"Acurácia KNN ponderado: {accuracy_score(y_test_d, y_pred_ponderado):.3f}")

print("\nPrevisões KNN normal:")
print(Counter(y_pred_normal))
print("\nPrevisões KNN ponderado:")
print(Counter(y_pred_ponderado))

## Resumo das Premissas

| Premissa | Descrição | Ação |
|----------|-----------|------|
| **Similaridade** | Pontos próximos pertencem à mesma classe | Garantir features relevantes |
| **Métrica de distância** | Define o conceito de proximidade | Escolher métrica adequada ao problema |
| **Normalização** | Features em escalas diferentes distorcem distâncias | Sempre normalizar/padronizar |
| **Valor de K** | Define viés vs variância | Testar diferentes valores (validação cruzada) |
| **Balanceamento** | Classes desbalanceadas enviesam predições | Ponderar ou rebalancear dados |

**Quando usar KNN:**
- Dados com fronteiras de decisão não-lineares
- Datasets pequenos a médios
- Quando interpretabilidade local é importante

**Quando evitar KNN:**
- Datasets muito grandes (custo computacional alto)
- Muitas dimensões (curse of dimensionality)
- Dados com muito ruído